<a href="https://colab.research.google.com/github/matthewlai12/ECE-Lab/blob/main/ECE449Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ECE 449 Lab 5: Matthew Lai
In this lab, our goal is to optimize our fuzzy tree from lab 4 so that it outperforms it. We will do so by using genetic algorithms to train our fuzzy system on.

##Preliminary Installations and Imports

In [87]:
!pip install scikit-fuzzy
!pip install EasyGA
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import EasyGA


##Section 1: Data Cleaning
In this section we will clean our datasets by clipping values less than 0 and greater to than 1 to be 0 and 1 respectively.

In [88]:
training_set = pd.read_csv("tipper_train.csv")
test_set = pd.read_csv("tipper_test.csv")

for dataset in [training_set, test_set]:
    # clip values between 0 and 1
    dataset.clip(lower=0, inplace=True)
    dataset.clip(upper=1, inplace=True)

##Section 2: Defining our Fuzzy System
In this section, we will be focused on defining antecedents and consequents. We will also define our genes.

###Antecedents:

In [89]:
# Function to define our antecedents
def antecedents():
  # Food Quality Antecedents
  temperature = ctrl.Antecedent(np.linspace(0, 10, 11),'temperature')
  flavor = ctrl.Antecedent(np.linspace(0, 10, 11),'flavor')
  portion_size = ctrl.Antecedent(np.linspace(0, 10, 11),'portion_size')
  # Service Quality Antecedents:
  attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11),'attentiveness')
  friendliness = ctrl.Antecedent(np.linspace(0, 10, 11),'friendliness')
  speed_of_service = ctrl.Antecedent(np.linspace(0, 10, 11),'speed_of_service')

  # #Antecedents for the second stage
  food_quality_input = ctrl.Antecedent(np.linspace(0, 10, 11),'food_quality_input')
  service_quality_input = ctrl.Antecedent(np.linspace(0, 10, 11),'service_quality_input')
  return temperature, flavor, portion_size, attentiveness, friendliness, speed_of_service, food_quality_input, service_quality_input

###Consequents:

In [90]:
# Function to define our consequents
def consequents():
  # Food Quality Consequents
  food_quality_output = ctrl.Consequent(np.linspace(0, 10, 11),'food_quality_output')
  # Service Quality Consequents
  service_quality_output = ctrl.Consequent(np.linspace(0, 10, 11),'service_quality_output')

  # Second Stage Consequents
  tip = ctrl.Consequent(np.linspace(0, 25, 26),'tip')
  return food_quality_output, service_quality_output, tip

###Genes:

In [91]:
def gene():

    poor = [np.random.uniform(0, 2 - 0.5), np.random.uniform(2 - 0.5, 2 + 0.5), np.random.uniform(2 + 0.5, 4)]
    average = [np.random.uniform(3, 5 - 0.5), np.random.uniform(5 - 0.5, 5 + 0.5), np.random.uniform(5 + 0.5, 7)]
    good = [np.random.uniform(6, 8 - 0.5), np.random.uniform(8 - 0.5, 8 + 0.5), np.random.uniform(8 + 0.5, 10)]

    chromosome = poor + average + good
    print("Chromosome: ", chromosome)
    return chromosome

###Rules:
We use the same rules as last lab.

In [92]:
# Function to define our rules
def rules(temperature, flavor, portion_size, attentiveness, friendliness, speed_of_service, food_quality_input, service_quality_input, food_quality_output, service_quality_output, tip):
  food_rule1 = ctrl.Rule(temperature['poor'] | flavor['poor'] | portion_size['poor'], food_quality_output['poor'])
  food_rule2 = ctrl.Rule(temperature['average'] & flavor['average'] & portion_size['average'], food_quality_output['average'])
  food_rule3 = ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['good'], food_quality_output['good'])

  service_rule1 = ctrl.Rule(attentiveness['poor'] | friendliness['poor'] | speed_of_service['poor'], service_quality_output['poor'])
  service_rule2 = ctrl.Rule(attentiveness['average'] & friendliness['average'] & speed_of_service['average'], service_quality_output['average'])
  service_rule3 = ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed_of_service['good'], service_quality_output['good'])

  tipping_rule1 = ctrl.Rule(food_quality_input['poor'] | service_quality_input['poor'], tip['poor'])
  tipping_rule2 = ctrl.Rule(food_quality_input['average'] & service_quality_input['average'], tip['average'])
  tipping_rule3 = ctrl.Rule(food_quality_input['good'] & service_quality_input['good'], tip['good'])

  return food_rule1, food_rule2, food_rule3, service_rule1, service_rule2, service_rule3, tipping_rule1, tipping_rule2, tipping_rule3

##Section 3: Building our Fuzzy System
In this section we will create our fuzzy system.

In [93]:
def setup_fuzzy_system(chromosome):

    temperature, flavor, portion_size, attentiveness, friendliness, speed_of_service, food_quality_input, service_quality_input = antecedents()
    food_quality_output, service_quality_output, tip = consequents()

    print("Chromosome: ", chromosome)
    universe = np.linspace(0, 10, 11)

    gene = chromosome[0]



    if type(chromosome) == list:
      a_pr, b_pr, c_pr = gene[0], gene[1], gene[2]
      a_avg, b_avg, c_avg = gene[3], gene[4], gene[5]
      a_gd, b_gd, c_gd = gene[6], gene[7], gene[8]

    else:
      a_pr, b_pr, c_pr = gene.value[0], gene.value[1], gene.value[2]
      a_avg, b_avg, c_avg = gene.value[3], gene.value[4], gene.value[5]
      a_gd, b_gd, c_gd = gene.value[6], gene.value[7], gene.value[8]

    temperature["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    temperature["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    temperature["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])


    flavor["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    flavor["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    flavor["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])


    portion_size["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    portion_size["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    portion_size["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])


    attentiveness["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    attentiveness["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    attentiveness["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])


    friendliness["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    friendliness["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    friendliness["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])



    speed_of_service["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    speed_of_service["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    speed_of_service["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])


    food_quality_input["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    food_quality_input["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    food_quality_input["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])

    service_quality_input["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    service_quality_input["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    service_quality_input["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])



    food_quality_output["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    food_quality_output["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    food_quality_output["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])



    service_quality_output["poor"] = fuzz.trimf(universe, [a_pr, b_pr, c_pr])
    service_quality_output["average"] = fuzz.trimf(universe, [a_avg, b_avg, c_avg])
    service_quality_output["good"] = fuzz.trimf(universe, [a_gd, b_gd, c_gd])



    tip["poor"] = fuzz.trimf(tip.universe, [a_pr, b_pr, c_pr])
    tip["average"] = fuzz.trimf(tip.universe, [a_avg, b_avg, c_avg])
    tip["good"] = fuzz.trimf(tip.universe, [a_gd, b_gd, c_gd])

    food_rule1, food_rule2, food_rule3, service_rule1, service_rule2, service_rule3, tipping_rule1, tipping_rule2, tipping_rule3 = rules(temperature, flavor, portion_size, attentiveness, friendliness, speed_of_service, food_quality_input, service_quality_input, food_quality_output, service_quality_output, tip)

    food_quality_ctrl = ctrl.ControlSystem([food_rule1, food_rule2, food_rule3])
    food_sim = ctrl.ControlSystemSimulation(food_quality_ctrl)

    service_quality_ctrl = ctrl.ControlSystem([service_rule1, service_rule2, service_rule3])
    service_sim = ctrl.ControlSystemSimulation(service_quality_ctrl)

    tip_ctrl = ctrl.ControlSystem([tipping_rule1, tipping_rule2, tipping_rule3])
    tip_sim = ctrl.ControlSystemSimulation(tip_ctrl)

    return food_sim, service_sim, tip_sim





##Section 4: Running our Simulations
In this section, we will use a combination of functions to run our fuzzy system.

###Simulation Function:

In [94]:
def execute_fuzzy_inference(food_sim, service_sim, tip_sim, inputs):

    # Simulation 1: Determining food qualtiy
    food_sim.input["temperature"] = inputs["temperature"]
    food_sim.input["flavor"] = inputs["flavor"]
    food_sim.input["portion_size"] = inputs["portion_size"]
    food_sim.compute()

    # Simulation 2: Determining service quality
    service_sim.input["attentiveness"] = inputs["attentiveness"]
    service_sim.input["friendliness"] = inputs["friendliness"]
    service_sim.input["speed_of_service"] = inputs["speed_of_service"]
    service_sim.compute()


    # Simulation 3: Determining tip amount
    tip_sim.input["food_quality_input"] = food_sim.output["food_quality_output"]
    tip_sim.input["service_quality_input"] = service_sim.output["service_quality_output"]
    tip_sim.compute()
    recommended_tip = tip_sim.output["tip"]
    print("Recommended Tip: ", recommended_tip)
    return recommended_tip*3

###Fitness Function

In [95]:
def fitness(chromosome):
    food_sim, service_sim, tip_sim = setup_fuzzy_system(chromosome)
    total_error = 0
    for index, row in training_set.iterrows():
        inputs = {
          "temperature": max(min(row["food temperature"]*10, max(chromosome[0].value)), min(chromosome[0].value)),
          "flavor": max(min(row["food flavor"]*10, max(chromosome[0].value)), min(chromosome[0].value)),
          "portion_size": max(min(row["portion size"]*10, max(chromosome[0].value)), min(chromosome[0].value)),
          "attentiveness": max(min(row["attentiveness"]*10, max(chromosome[0].value)), min(chromosome[0].value)),
          "friendliness": max(min(row["friendliness"]*10, max(chromosome[0].value)), min(chromosome[0].value)),
          "speed_of_service": max(min(row["speed of service"]*10, max(chromosome[0].value)), min(chromosome[0].value))
        }
        actual_tip = row["tip"]
        predicted_tip = execute_fuzzy_inference(food_sim, service_sim, tip_sim, inputs)
        error = abs(actual_tip - predicted_tip)
        total_error += error
        return total_error

###GA Function:

In [96]:
import os
import EasyGA

# Ensure database is removed
while True:
    if os.path.exists("database.db"):
        os.remove("database.db")
    else:
        break


# Define the genetic algorithm
ga = EasyGA.GA()
ga.gene_impl = lambda: gene()
ga.chromosome_length = 1  # genes in a chromosome
ga.population_size = 20  # number of chromosomes in the population
ga.target_fitness_type = "min"
ga.generation_goal = 2  # number of generations to run
ga.fitness_function_impl = fitness

# Run the genetic algorithm
ga.evolve()
ga.print_best_chromosome()

best_chromosome = input("Input the best chromosome values above, make sure the input is a string with comma seperated values inside the chromosome")

while(len(list(map(float, (item.strip(' ') for item in list(best_chromosome.strip('[]').split(',')))))) != 9):
  best_chromosome = input('Invalid input! Try again: ')

chromosome_values = list(map(float, (item.strip(' ') for item in list(best_chromosome.strip('[]').split(',')))))

print('\nChromosome values')
food_sim, service_sim, tip_sim = setup_fuzzy_system(chromosome=[chromosome_values])


Chromosome:  [1.37231970922485, 1.6915543025249073, 3.6213629633134654, 3.943131765135691, 5.307532415181967, 5.591044698152918, 6.475387920153994, 8.087906032123911, 9.252568120995832]
Chromosome:  [0.8193600389368019, 2.40343292909715, 3.049137502973756, 3.1543169846532226, 5.023329044536812, 6.410930500047901, 6.275416397677081, 8.140773527591493, 8.942645043953886]
Chromosome:  [0.5715363721939335, 1.6672510969525336, 3.4353517336439654, 3.4201228456416324, 4.841656026438157, 6.847805407632236, 6.140110024195794, 7.828231313989757, 9.782153760224817]
Chromosome:  [1.1289319878692674, 1.6524781132550483, 3.209070039550511, 3.313968834528687, 4.7343665113032145, 5.983243666049794, 6.737982309543411, 8.379922543792516, 8.898804506726107]
Chromosome:  [1.4042847271931325, 2.0551581530626506, 2.689797138326716, 3.2405074734811548, 4.549657620521203, 6.079212056455524, 6.1055120594222, 8.412933257939343, 9.49605225512692]
Chromosome:  [0.8116893843130814, 2.010063466629898, 2.75323022666

###Run Function:

In [99]:
10# Function to validate inputs
def get_valid_input(prompt, min_val=0, max_val=10):
    while True:
        try:
            value = float(input(prompt))
            if min_val <= value <= max_val:
                return value
            else:
                print(f"Please enter a value between {min_val} and {max_val}.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

# Start loop
while True:
    # Get the inputs
    temperature_val = get_valid_input("Enter temperature (0-10): ")
    flavor_val = get_valid_input("Enter flavor (0-10): ")
    portion_size_val = get_valid_input("Enter portion size (0-10): ")

    attentiveness_val = get_valid_input("Enter attentiveness (0-10): ")
    friendliness_val = get_valid_input("Enter friendliness (0-10): ")
    speed_of_service_val = get_valid_input("Enter speed of service (0-10): ")

    # Set the inputs
    food_sim.input["temperature"] = max(min(temperature_val, max(chromosome_values)), min(chromosome_values))
    food_sim.input["flavor"] = max(min(flavor_val, max(chromosome_values)), min(chromosome_values))
    food_sim.input["portion_size"] = max(min(portion_size_val, max(chromosome_values)), min(chromosome_values))

    # Compute food_quality_sim
    food_sim.compute()
    food_quality_output = food_sim.output['food_quality_output']

    # Set the inputs
    service_sim.input["attentiveness"] = max(min(attentiveness_val, max(chromosome_values)), min(chromosome_values))
    service_sim.input["friendliness"] = max(min(friendliness_val, max(chromosome_values)), min(chromosome_values))
    service_sim.input["speed_of_service"] = max(min(speed_of_service_val, max(chromosome_values)), min(chromosome_values))

    # Compute service_quality_sim
    service_sim.compute()
    service_quality_output = service_sim.output['service_quality_output']

    # Set inputs based on results above
    tip_sim.input["food_quality_input"] = max(min(food_sim.output["food_quality_output"], max(chromosome_values)), min(chromosome_values))
    tip_sim.input["service_quality_input"] = max(min(service_sim.output["service_quality_output"], max(chromosome_values)), min(chromosome_values))

    # Compute tip output
    tip_sim.compute()
    tip_output = tip_sim.output['tip']*3

    # Print results
    print("\nResults:")
    print(f"Food Quality: {food_quality_output:.2f}")
    print(f"Service Quality: {service_quality_output:.2f}")
    print(f"Tip Recommendation: {tip_output:.2f}\n")

    # Continue or break the loop
    continue_simulation = input("Do you want to perform another simulation? (yes/no): ").strip().lower()
    if continue_simulation != "yes":
        print("Exiting simulation.")
        break


Enter temperature (0-10): 10
Enter flavor (0-10): 10
Enter portion size (0-10): 10
Enter attentiveness (0-10): 10
Enter friendliness (0-10): 10
Enter speed of service (0-10): 10

Results:
Food Quality: 7.93
Service Quality: 7.93
Tip Recommendation: 23.73

Do you want to perform another simulation? (yes/no): yes
Enter temperature (0-10): 0
Enter flavor (0-10): 0
Enter portion size (0-10): 0
Enter attentiveness (0-10): 0
Enter friendliness (0-10): 0
Enter speed of service (0-10): 0

Results:
Food Quality: 1.51
Service Quality: 1.51
Tip Recommendation: 4.98

Do you want to perform another simulation? (yes/no): yes
Enter temperature (0-10): 5
Enter flavor (0-10): 5
Enter portion size (0-10): 5
Enter attentiveness (0-10): 5
Enter friendliness (0-10): 5
Enter speed of service (0-10): 5

Results:
Food Quality: 4.97
Service Quality: 4.97
Tip Recommendation: 14.92

Do you want to perform another simulation? (yes/no): yes
Enter temperature (0-10): 0
Enter flavor (0-10): 5
Enter portion size (0-1

##Section 5: Conclusion
We will compare our results between the first and second time we used fuzzy systems now:

When all inputs are consistently the same, we will compare the tip value:

| Value | Lab 4 | Lab 5 |
|----------|----------|----------|
| 10   | 15.28 | 23.73 |
| 0   | 9.72 | 4.98 |
| 5   | 12.50  | 14.92 |

In our case here, we can see that when the quality was higher it resulted in a better tip, and when quality was lower it resulted in a lower tip.
When our tip was right in the middle, it was close to the middle value in both.
Our key difference is that we will notice a a much better spread of values, leading us to see that the GA model was indeed better and more accurate.


###References:
- ECE 449 Lab 5 Manual
- ECE 449 Lab 5 Presentation
- ECE 449 Slides posted by Dr. Musilek
- ChatGPT: In this lab ChatGPT was used to help fix errors in my loop as well as to help validate inputs.